# Ejecta model

TARDIS requires a model of the homologously expanding ejecta in order to run a simulation. The ejecta is modeled as concentric 1 dimensional radial cells. Each cell contins information the velocity structure, abundances, density, and a radiation field.
#This information is stored in TARDIS [config](link). 
The information of these properties can be stored as classes and all of them inherit from a BaseProperty to facilitate the interaction. BaseProperty handles the initailization of the property's time and initial time (time_0) with which most properties build on.             
##  Model grid
The first thing needed in order to have a useful ejecta model is a discreet space grid. 
What defines the spatial grid in TARDIS is the velocity grid. This dependance on velocity also entails a dependance with time.
### Velocity Class
For a basic grid inner and outer velocities are needed. Each concentric cell has an inner and outer velocity, and inner_velocity and outer_velocity are arrays composed of these velocities.
The velocity class stores velocities and velocities-related quantities (such as cell radius and volume). To start a Velocity class we need `inner_velocity` and an `outer_velocity` arrays, optional arguments are `time_0` which fallback to 0 seconds and `time` which is recomended to have singificant values for radii and volume.


In [1]:
from tardis.model._generic_model import *

/home/jordi/conda/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [2]:
velocity_inner = [i for i in range(9000, 13000, 500)] * u.km / u.s
velocity_outer = [i for i in range(9500, 13500, 500)] * u.km / u.s
print(velocity_inner,'\n',velocity_outer)

[ 9000.  9500. 10000. 10500. 11000. 11500. 12000. 12500.] km / s 
 [ 9500. 10000. 10500. 11000. 11500. 12000. 12500. 13000.] km / s


In [3]:
velocity = Velocity(velocity_inner,velocity_outer)


no `time` provided, `time_0` will be used


Quantities are converted to CGS and can be easily accessed:

In [4]:
velocity.middle

<Quantity [9.250e+08, 9.750e+08, 1.025e+09, 1.075e+09, 1.125e+09,
           1.175e+09, 1.225e+09, 1.275e+09] cm / s>

By not providing a time for the class some quantities don't make sense:

In [7]:
velocity.inner_radius

<Quantity [7.7760e+13, 8.2080e+13, 8.6400e+13, 9.0720e+13, 9.5040e+13,
           9.9360e+13, 1.0368e+14, 1.0800e+14] cm>

`time` can be modified:

In [6]:
velocity.time = 1 * u.d
velocity.time

<Quantity 1. d>

Quantities stored in Velocity are:
 - Velocity.inner 
 - Velocity.outer
 - Velocity.middle
 - Velocity.inner_radius
 - Velocity.middle_radius
 - Velocity.outer_radius
 - Velocity.volume
 - Velocity.time_0
 - Velocity.time
 - Velocity.number_of_cells

The number of cells is defined by the length of the input velocity arrays

In [8]:
velocity.number_of_cells

8

### Density




### Abundance
Explain
Example
### Radiation Field
Explain
Example
## Generic[Ejecta]Model
A more flexible implementation of the TARDIS ejecta model is the Generic[Ejecta]Model. 
What does Generic model is and what it does with data.

Explain what you can do here but not in TARDIS
Start model and different times using above properties 
Explain that more properties can be added as long as they use the basxi structure of BaseProperty
## TARDIS Ejecta Model
Explain what happens with v_boundary and that it can't change.